In [1]:
import nifgen
import time
import numpy as np
import os
import math

In [3]:
# Outputs a sine wave
with nifgen.Session("Dev1") as session:
    session.output_mode = nifgen.OutputMode.FUNC
    session.configure_standard_waveform(waveform = nifgen.Waveform.SINE, amplitude=1.0, frequency = 100000)
    with session.initiate():
        time.sleep(5)

In [9]:
# Chunking waveform for streaming

# Raw waveform
waveform = r"C:\Users\max\Desktop\NanoStride\waveform_output.bin"

# Determining allocation size
def allocation_size(waveform):
    total_size_bytes = os.path.getsize(waveform)
    max_chunk_size = 10 * 1024 * 1024 # 200MB

    valid_chunks = set()
    for i in range(1, int(math.isqrt(total_size_bytes))+1):
        if total_size_bytes % i == 0:
            complement = total_size_bytes // i
            if i <= max_chunk_size:
                valid_chunks.add(i)
            if complement <= max_chunk_size:
                valid_chunks.add(complement)
    return(max(valid_chunks) if valid_chunks else None)

# Setting waveform handle and waveform size
waveform_size = allocation_size(r"C:\Users\max\Desktop\NanoStride\waveform_output.bin")


In [10]:
# Loading first chunk for test
first_chunk = open(waveform, "rb").read(allocation_size(waveform))


In [11]:
len(first_chunk)

9536000

In [7]:
with nifgen.Session("Dev1") as session:
    session.clear_arb_memory()

In [ ]:
# Testing abitrary waveform

#waveform = np.fromfile(r"C:\Users\max\Desktop\NanoStride\waveform_output.bin", dtype=np.float64)
#waveform_handle = session.create_waveform_from_file_f64(r'C:\Users\max\Desktop\NanoStride\waveform_output.bin', byte_order=nifgen.ByteOrder.LITTLE)


with nifgen.Session("Dev1") as session:
    session.output_mode = nifgen.OutputMode.ARB
    session.arb_sample_rate = 25e6

    num_samples = 100
    waveform = np.sin(np.linspace(0, 2 * np.pi, num_samples)).tolist()
    waveform_handle = session.create_waveform(waveform)
    #waveform_handle = session.streaming_waveform_handle()
    #session.channels["0"].arb_waveform = waveform_handle

    with session.initiate():
        time.sleep(5)

In [24]:
def stream_waveform(waveform_path, chunk_size_bytes):
    with open(waveform_path, "rb") as f:
        first_bytes = f.read(chunk_size_bytes)

    # Convert raw bytes to float64, then to float32
    first_data = np.frombuffer(first_bytes, dtype='<f8').astype(np.float32)

    with nifgen.Session("Dev1") as session:
        session.output_mode = nifgen.OutputMode.SCRIPT
        session.arb_sample_rate = 25e6

        waveform_name = "streamwave"
        num_samples = len(first_data)


        # ✅ Allocate a named waveform
        session.allocate_named_waveform(waveform_name, num_samples)

        # ✅ Write the waveform using the name (not handle)
        session.write_waveform(waveform_name, first_data.tolist())

        # ✅ Write a script that references the named waveform
        session.write_script(f"""
        script main
            repeat 1000
                generate {waveform_name}
            end repeat
        end script
        """)


        # ✅ Start generation
        with session.initiate():
            session.wait_until_done()

if __name__ == "__main__":
    chunk_size_bytes = 64000 * 1024  # 64 KB
    waveform_path = r"C:\Users\max\Desktop\NanoStride\waveform_output.bin"
    stream_waveform(waveform_path, chunk_size_bytes)


DriverError: -1074118637: Maximum time exceeded before the operation completed.

In [40]:
def stream_full_waveform(waveform_path):

    # Determine chunk size and number of chunks
    chunk_size_bytes = allocation_size(waveform_path)
    total_size_bytes = os.path.getsize(waveform_path)
    num_chunks = total_size_bytes // chunk_size_bytes

    with open(waveform_path, "rb") as f:
        for _ in range(num_chunks):
            print(_)
            chunk = f.read(chunk_size_bytes)

            # Ensure chunk size is a multiple of 8 bytes (float64)
            valid_bytes = len(chunk) - (len(chunk) % 8)
            chunk = chunk[:valid_bytes]

            data = np.frombuffer(chunk, dtype='<f8').astype(np.float32)

            with nifgen.Session("Dev1") as session:
                session.output_mode = nifgen.OutputMode.SCRIPT
                session.arb_sample_rate = 25e6

                waveform_name = "streamwave"
                num_samples = len(data)


                # ✅ Allocate a named waveform
                session.allocate_named_waveform(waveform_name, num_samples)

                # ✅ Write the waveform using the name (not handle)
                session.write_waveform(waveform_name, data.tolist())

                # ✅ Write a script that references the named waveform
                session.write_script(f"""
                script main
                    repeat 1
                        generate {waveform_name}
                    end repeat
                end script
                """)


                # ✅ Start generation
                with session.initiate():
                    session.wait_until_done()

if __name__ == "__main__":
    chunk_size_bytes = 64000 * 1024  # 64 KB
    waveform_path = r"C:\Users\max\Desktop\NanoStride\waveform_output.bin"
    stream_full_waveform(waveform_path)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124


In [36]:
# Raw waveform path
waveform_path = r"C:\Users\max\Desktop\NanoStride\waveform_output.bin"

# Determine chunk size and number of chunks
chunk_size_bytes = allocation_size(waveform_path)
total_size_bytes = os.path.getsize(waveform_path)
num_chunks = total_size_bytes // chunk_size_bytes

# Read and print each chunk
with open(waveform_path, "rb") as f:
    for _ in range(num_chunks):
        chunk = f.read(chunk_size_bytes)

        # Ensure chunk size is a multiple of 8 bytes (float64)
        valid_bytes = len(chunk) - (len(chunk) % 8)
        chunk = chunk[:valid_bytes]

        data = np.frombuffer(chunk, dtype='<f8').astype(np.float32)
        print(data)


[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -

In [49]:
def stream_full_waveform(waveform_path):
    # Determine chunk size and number of chunks
    chunk_size_bytes = allocation_size(waveform_path)
    total_size_bytes = os.path.getsize(waveform_path)
    num_chunks = total_size_bytes // chunk_size_bytes
    bytes_streamed = 0  # Track total bytes streamed

    with open(waveform_path, "rb") as f:
        for _ in range(num_chunks):
            print(_)
            chunk = f.read(chunk_size_bytes)
            bytes_streamed += len(chunk)

            # Ensure chunk size is a multiple of 8 bytes (float64)
            valid_bytes = len(chunk) - (len(chunk) % 8)
            chunk = chunk[:valid_bytes]

            data = np.frombuffer(chunk, dtype='<f8').astype(np.float32)

            with nifgen.Session("Dev1") as session:
                session.output_mode = nifgen.OutputMode.SCRIPT
                session.arb_sample_rate = 25e6

                waveform_name = "streamwave"
                num_samples = len(data)

                # ✅ Allocate a named waveform
                session.allocate_named_waveform(waveform_name, num_samples)

                # ✅ Write the waveform using the name (not handle)
                session.write_waveform(waveform_name, data.tolist())

                # ✅ Write a script that references the named waveform
                session.write_script(f"""
                script main
                    repeat 1
                        generate {waveform_name}
                    end repeat
                end script
                """)

                # ✅ Start generation
                with session.initiate():
                    session.wait_until_done()

    print(f"Total bytes streamed: {bytes_streamed}")
    print(f"Total file size     : {total_size_bytes}")


In [50]:
waveform_path = r"C:\Users\max\Desktop\NanoStride\waveform_output.bin"
stream_full_waveform(waveform_path)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
Total bytes streamed: 1192000000
Total file size     : 1192000000
